In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import cv2
from skimage import io
import requests
import pandas as pd
import feature_extractor , config_utils

fe=feature_extractor.FeatureExtractor()

In [250]:

df=pd.read_csv(config_utils.url_filtered_path,usecols=['ImageID','OriginalURL','Tags', 'Author','Title'])



In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Function to extract features from a URL
def extract_features_from_url(url):
    try:
        return url, fe.get_features_from_link(url)
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return None

# Number of threads you want to use for parallel processing
num_threads = 4  # You can adjust this number

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    pbar = tqdm(total=len(df[10000:20000]), desc="Extracting Features")

    # Submit tasks for each URL to be processed
    futures = [executor.submit(extract_features_from_url, url) for url in df['OriginalURL'][10000:20000]]

    extracted_features95 = {}

    # Iterate over completed futures
    for future in futures:
        result = future.result()
        if result is not None:
            url, features = result
            extracted_features95[url] = features
        pbar.update(1)

# Create a list of dictionaries excluding None values
data = [{'OriginalURL': url, 'FeatureVector': json.dumps(feature_vector.tolist())} for url, feature_vector in extracted_features95.items()]



In [11]:
import json
# Create a list of dictionaries excluding None values
data = [{'OriginalURL': url, 'FeatureVector': json.dumps(feature_vector.tolist())} for url, feature_vector in extracted_features95.items()]

df10 = pd.DataFrame(data)
df10.to_csv(config_utils.data_10k,index=False)

In [154]:
df10=pd.read_csv(config_utils.data_10k)
df10['FeatureVector'] = df100['FeatureVector'].apply(lambda x: np.array(json.loads(x)))


In [156]:
df10.head()

,OriginalURL,FeatureVector
0,https://c8.staticflickr.com/8/7484/15947609787...,"[0.0, 0.0037798460107296705, 0.030225254595279..."
1,https://farm7.staticflickr.com/6224/6342312314...,"[0.0, 0.0, 0.0, 0.011376092210412025, 0.0, 0.0..."
2,https://c3.staticflickr.com/3/2548/3886222799_...,"[0.0, 0.022251930087804794, 0.0060163619928061..."
3,https://c3.staticflickr.com/4/3250/5754857912_...,"[0.022978525608778, 0.0, 0.0, 0.02252731099724..."
4,https://farm5.staticflickr.com/7012/6715043879...,"[0.0, 0.0, 0.07514772564172745, 0.051324415951..."


In [176]:
df10.FeatureVector[0].shape

(4096,)

In [12]:
df5 = pd.read_csv(config_utils.data_5k)
df5['FeatureVector'] = df5['FeatureVector'].apply(lambda x: np.array(json.loads(x)))
df5.head()

,OriginalURL,FeatureVector
0,https://farm5.staticflickr.com/8637/1661040770...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030451796948..."
1,https://c3.staticflickr.com/2/1157/1462090204_...,"[[0.0, 0.010526580736041069, 0.001877285074442..."
2,https://c8.staticflickr.com/5/4023/4206922500_...,"[[0.0, 0.0, 0.03080959990620613, 0.0, 0.0, 0.0..."
3,https://c3.staticflickr.com/9/8013/7414044012_...,"[[0.0, 0.0, 0.055793918669223785, 0.0, 0.0, 0...."
4,https://c2.staticflickr.com/9/8312/8039418292_...,"[[0.0, 0.003233593422919512, 0.0, 0.0213613975..."


In [177]:
big_df = pd.concat([df10, df5], axis=0)


In [179]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14488 entries, 0 to 9690
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalURL    14488 non-null  object
 1   FeatureVector  14488 non-null  object
dtypes: object(2)
memory usage: 339.6+ KB


In [182]:
data=big_df['FeatureVector'].values


In [183]:
data.shape

(14488,)

In [185]:

flattened_data = np.array([x.flatten() for x in data])


In [187]:
from sklearn.decomposition import PCA

n_components = 786
pca = PCA(n_components=n_components)

# Fit the PCA model to your data
pca.fit(flattened_data)


PCA(n_components=786)

In [194]:
from joblib import dump
model_filename = "pca_model20k.pkl"
dump(pca, model_filename)



['pca_model20k.pkl']

In [244]:
big_df['FeatureVector'] = big_df['FeatureVector'].apply(lambda x: pca.transform(x.reshape(1, -1))[0])


In [254]:
final = pd.merge(df, big_df, on='OriginalURL', how='inner')


In [261]:
final.head()

,OriginalURL,ImageID,Author,Title,Tags,FeatureVector
0,https://c8.staticflickr.com/8/7484/15947609787...,43971fd64594d6ef,Andres Perez,La #selfie de mi #amor con el #camilin #sobrino,['Toddler'],"[-0.21065448338646245, 0.08222337240108045, -0..."
1,https://farm7.staticflickr.com/6224/6342312314...,6a506a3a53143fa6,Ron Cogswell,Potomac Eagle Train -- North of Romney Saturda...,"['Screenshot', 'Train']","[0.07606110373707096, 0.005661928558624046, 0...."
2,https://c3.staticflickr.com/3/2548/3886222799_...,57407b991842716a,Christian Heilmann,xml being found is a problem?,['Screenshot'],"[-0.006623505660583157, -0.166815973533954, -0..."
3,https://c3.staticflickr.com/4/3250/5754857912_...,4b90b2161756f72a,SA-Venues.com,Flowers beside the lake - Green Point Park,"['Lake', 'Plant', 'Tree', 'Flower']","[0.20381353141968522, 0.0442960083283837, 0.10..."
4,https://farm5.staticflickr.com/7012/6715043879...,7630b725776dc6c4,baligraph,"Travel to East Java , a Tropical Paradise in T...","['Computer', 'Screenshot', 'Food']","[-0.007265260920830729, -0.230973277233518, -0..."


In [273]:
final['FeatureVector'] = final['FeatureVector'].apply(lambda x: json.dumps(x.tolist()))


In [276]:
final.to_csv(config_utils.final_with_feature,index=False)